In [1]:
import logging
import requests
import os
import json
from datetime import datetime, timezone 
from urllib.parse import urljoin, urlencode

/Users/sarvagra/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# logging configuration
logging_dir="../logs"
os.makedirs(logging_dir, exist_ok=True)
logging.basicConfig(filename=os.path.join(logging_dir,"scraped_discourse_content.log"), level=logging.DEBUG, format='%(asctime)s %(message)s', force=True)
logging.info("logging is configured")

In [3]:
# configuration 
logging.info("setting up configuration variables ...")
try:
    base_url = "https://discourse.onlinedegree.iitm.ac.in/"
    category = "courses/tds-kb"
    category_id = 34

    # set start and end dates for topics to fetch
    start_date = "2025-01-01" 
    end_date = "2025-04-14"   

    cookie_str ="_t=UhgFW7puAicWwzys5K9x4XXCeIbAiuOffLbEHteeJpOcxWknvKkpxkuCXAJDDj%2BCEBdIzV87gyPbeEKqYFM%2BTu4Nbus%2FZdJcHa9Ay9ME5NIKAq16wM4TaRgJK3Mo0cQsXXlGX2fdCT5D%2B5gbq%2FzE1oudFCFWqvejbRP9Lwr6gAuXOwvecipvP368M2ahMi7jBt8DW1M7g5AT75IMk4P9Pq2lgD4Wn2XFsNlDhSZC3a1bEBPd%2BHIh1GggvwXvG0kwR7SDQUqR26E9u8TK%2FO0UtXhvJPkGo1JIDK0t1agSkN%2BwlZmvtOKmRDGJsxZ%2FjnBR--efLSXFn%2BwaEvunu1--7%2B89YY7QWAMaRwNWi9a29Q%3D%3D; _forum_session=TtAj8x2p%2BWKovx0VoSopoqABmRXBQMMlCSjlvNY5Rda3Kllr4hE%2FwK9Dl8jwSmFcNhQlivTIOph3UZ3XTwi6duFXv2xDTJQmDFBPkIUmu187OPqj%2FtQ2hGXETtSR5VL8HJ1zDMFxoAvWmxo8gVsPkqCBI%2BcPaRV7ov5L7v%2FQiQUeRDRtKdJ0%2B9aRR0zw1qnnTYPFI7XOPXhnZjxGljbDlUAkCFH56QW5FoduR%2BK3IPGEbHjbcaISLF91OngGq9Swk%2FYuO9LZ9e2qRDItXzg0mEvZ01KGmQ%3D%3D--lBY22MvA7ZuSLszQ--QiC%2FbwHvWPWAH7XSYaiGEw%3D%3D" 
    
    out_dir = "scraped_data_discourse_page" 
    post_batch_id_size = 50 
    breakpoint = 5 # New configuration for breaking loop
    logging.info("configuration variables set successfully.")

except:
    logging.error("Error in configuration. Please check the configuration variables.")
    raise

In [4]:
# Parse a raw cookie string as dictionary.

logging.info("Parsing cookie string ...")
def parse_cookie_string(raw_cookie_string):
    try:
        cookies = {}
        if not raw_cookie_string.strip():
            print("Warning: RAW_COOKIE_STRING is empty. Requests might fail if authentication is needed.")
            return cookies
        for cookie_part in raw_cookie_string.strip().split(";"):
            if "=" in cookie_part:
                key, value = cookie_part.strip().split("=", 1)
                cookies[key] = value
        logging.info("cookie string parsed successfully.")
        return cookies
    
    except:
        logging.error("error in parsing cookie string")



In [5]:
# Fetche topic IDs from a specific category within a date range.
def get_topic_ids(base_url, category, category_id, start_date, end_date, cookies):
    url = urljoin(base_url, f"c/{category}/{category_id}.json")
    topic_ids = []
    page = 0

    start_dt_naive = datetime.fromisoformat(start_date + "T00:00:00")
    start_dt = start_dt_naive.replace(tzinfo=timezone.utc)
    end_dt_naive = datetime.fromisoformat(end_date + "T23:59:59.999999")
    end_dt = end_dt_naive.replace(tzinfo=timezone.utc)

    print(f"Fetching topic IDs from category between {start_dt} and {end_dt}...")

    # Variables for the new loop break condition
    consecutive_pages_with_no_new_unique_topics = 0
    last_known_unique_topic_count = 0

    while True:
        paginated_url = f"{url}?page={page}"
        try:
            response = requests.get(paginated_url, cookies=cookies, timeout=30)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Failed to fetch page {page}: {e}")
            break

        try:
            data = response.json()
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from page {page}. Content: {response.text[:200]}...")
            break

        topics_on_page = data.get("topic_list", {}).get("topics", [])

        if not topics_on_page:
            print(f"No more topics found on page {page} (API returned empty list).")
            break # Primary stop condition: API says no more topics on this page

        # Store current number of unique topics before processing this page
        # This helps check if *this specific page fetch* added anything new
        count_before_processing_page = len(set(topic_ids))

        for topic in topics_on_page:
            created_at_str = topic.get("created_at")
            if created_at_str:
                try:
                    created_date = datetime.fromisoformat(created_at_str.replace("Z", "+00:00"))
                except ValueError:
                    print(f"Warning: Could not parse date '{created_at_str}' for topic ID {topic.get('id')}")
                    continue

                if start_dt <= created_date <= end_dt:
                    topic_ids.append(topic["id"]) # Add ID, will be deduped later for count

        current_unique_topic_count = len(set(topic_ids))

        if topics_on_page and current_unique_topic_count == count_before_processing_page :
            # This means the current page had topics, but none of them were new *and* within the date range,
            # or all topics fetched from this page were duplicates of ones already in topic_ids from *previous pages*.
            # For the staleness check, we care if the overall unique set isn't growing.
             pass # Handled by the check below using last_known_unique_topic_count

        # Staleness check: Has the *total* number of unique topics found stopped growing?
        if current_unique_topic_count == last_known_unique_topic_count and topics_on_page:
            # topics_on_page is checked to ensure we don't increment if an empty page was returned (which is a valid end)
            consecutive_pages_with_no_new_unique_topics += 1
            print(f"Page {page} did not yield any new unique topics. Consecutive stale pages: {consecutive_pages_with_no_new_unique_topics}.")
        else:
            consecutive_pages_with_no_new_unique_topics = 0 # Reset if new unique topics were found

        last_known_unique_topic_count = current_unique_topic_count

        if consecutive_pages_with_no_new_unique_topics >= breakpoint:
            print(f"No new unique topics found for {breakpoint} consecutive pages. Assuming end of relevant category listing.")
            break

        # Original secondary stop condition (heuristic)
        more_topics_url = data.get("topic_list", {}).get("more_topics_url")
        if not more_topics_url:
            # This typically means it's the last page.
            # The condition `len(topics_on_page) < 30` was a heuristic for when more_topics_url might be missing
            # but the page wasn't full. If more_topics_url is definitively gone, it's a strong signal.
            print(f"No 'more_topics_url' indicated on page {page}. Assuming this is the last page of topics.")
            break
        
        print(f"Fetched page {page}, {len(topics_on_page)} topics on page. Total unique topics found so far: {current_unique_topic_count}. Continuing...")
        page += 1


    final_unique_topic_ids = list(set(topic_ids)) # Deduplicate
    print(f"Total unique topics found in timeframe: {len(final_unique_topic_ids)}")
    return final_unique_topic_ids


In [6]:
# Fetches the full topic JSON, including all posts by handling pagination.
def get_full_topic_json(base_url, topic_id, cookies):
    initial_topic_url = urljoin(base_url, f"t/{topic_id}.json")
    print(f"Fetching initial data for topic {topic_id} from {initial_topic_url}")

    try:
        response = requests.get(initial_topic_url, cookies=cookies, timeout=30)
        response.raise_for_status()
        topic_data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch initial topic data for {topic_id}: {e}")
        return None
    except json.JSONDecodeError:
        print(f"Failed to decode initial JSON for topic {topic_id}. Content: {response.text[:200]}...")
        return None

    post_stream = topic_data.get("post_stream")
    if not post_stream or "stream" not in post_stream or "posts" not in post_stream:
        print(f"Error: 'post_stream' not found or incomplete in topic {topic_id}. Skipping post fetching.")
        return topic_data

    all_post_ids_in_stream = post_stream.get("stream", [])
    loaded_post_ids = {post["id"] for post in post_stream.get("posts", [])}

    all_post_ids_in_stream = [pid for pid in all_post_ids_in_stream if pid is not None]

    missing_post_ids = [pid for pid in all_post_ids_in_stream if pid not in loaded_post_ids]

    print(f"Topic {topic_id}: Total posts in stream: {len(all_post_ids_in_stream)}, Initially loaded: {len(loaded_post_ids)}, Missing: {len(missing_post_ids)}")

    if not missing_post_ids:
        print(f"All posts for topic {topic_id} already loaded in initial fetch.")
        return topic_data

    fetched_additional_posts = []
    for i in range(0, len(missing_post_ids), post_batch_id_size):
        batch_ids = missing_post_ids[i:i + post_batch_id_size]

        query_params = [("post_ids[]", pid) for pid in batch_ids]
        posts_url = urljoin(base_url, f"t/{topic_id}/posts.json")

        print(f"Fetching batch of {len(batch_ids)} posts for topic {topic_id} (IDs: {batch_ids[0]}...{batch_ids[-1]})")

        try:
            batch_response = requests.get(posts_url, params=query_params, cookies=cookies, timeout=60)
            batch_response.raise_for_status()
            batch_data = batch_response.json()

            if isinstance(batch_data, list):
                 fetched_additional_posts.extend(batch_data)
            elif "post_stream" in batch_data and "posts" in batch_data["post_stream"]:
                fetched_additional_posts.extend(batch_data["post_stream"]["posts"])
            elif "posts" in batch_data and isinstance(batch_data["posts"], list):
                 fetched_additional_posts.extend(batch_data["posts"])
            else:
                print(f"Warning: Unexpected JSON structure for post batch in topic {topic_id}. Data: {str(batch_data)[:200]}...")

        except requests.exceptions.RequestException as e:
            print(f"Failed to fetch post batch for topic {topic_id} (IDs: {batch_ids}): {e}")
        except json.JSONDecodeError:
            print(f"Failed to decode JSON for post batch in topic {topic_id}. Response: {batch_response.text[:200]}...")

    if fetched_additional_posts:
        print(f"Successfully fetched {len(fetched_additional_posts)} additional posts for topic {topic_id}.")
        existing_posts_in_topic_data = {post['id']: post for post in topic_data["post_stream"]["posts"]}
        for post in fetched_additional_posts:
            if post['id'] not in existing_posts_in_topic_data:
                topic_data["post_stream"]["posts"].append(post)
                existing_posts_in_topic_data[post['id']] = post

        post_id_to_post_map = {post['id']: post for post in topic_data["post_stream"]["posts"]}

        sorted_posts = []
        for post_id_val in all_post_ids_in_stream: # Renamed post_id to post_id_val to avoid conflict
            if post_id_val in post_id_to_post_map:
                sorted_posts.append(post_id_to_post_map[post_id_val])

        topic_data["post_stream"]["posts"] = sorted_posts
        print(f"Topic {topic_id}: Final post count in JSON: {len(topic_data['post_stream']['posts'])}")

    return topic_data


In [7]:
# Saves the topic JSON data to a file.
def save_topic_json(topic_id, json_data, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    filepath = os.path.join(output_dir, f"topic_{topic_id}.json")
    try:
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        # print(f"Successfully saved topic {topic_id} to {filepath}") # Reduced verbosity
    except IOError as e:
        print(f"Error saving topic {topic_id} to {filepath}: {e}")

In [8]:
def main():
    """Main function to orchestrate the downloading process."""
    logging.info("script started")
    cookies = parse_cookie_string(cookie_str)
    if not cookies and base_url != "https://meta.discourse.org/":
        logging.warning("Running without cookies. This may fail for private forums or specific content.")

    topic_ids = get_topic_ids(
        base_url,
        category,
        category_id,
        start_date,
        end_date,
        cookies
    )

    if not topic_ids:
        print("No topic IDs found for the given criteria. Exiting.")
        return

    total_topics = len(topic_ids)
    success_downloads = 0
    failed_topic_ids = []

    print(f"\nStarting download of {total_topics} topics...\n")

    for i, topic_id in enumerate(topic_ids, 1):
        print(f"--- [{i}/{total_topics}] Processing topic ID: {topic_id} ---")
        topic_json_data = get_full_topic_json(base_url, topic_id, cookies)
        if topic_json_data:
            save_topic_json(topic_id, topic_json_data, out_dir)
            success_downloads += 1
        else:
            logging.warning("Failed to get complete data for topic %d.", topic_id)
            failed_topic_ids.append(topic_id)
        # logging.info("--- Finished processing topic ID: %d ---", topic_id)

    logging.info("\n========= SUMMARY =========")
    logging.info(f"Total topics identified: {total_topics}")
    logging.info(f"Successfully downloaded full data for: {success_downloads} topics")
    logging.info(f"Failed to download/process: {len(failed_topic_ids)} topics")
    if failed_topic_ids:
        logging.info("Failed topic IDs: %s", failed_topic_ids)
    logging.info("Downloaded files are in: %s", os.path.abspath(out_dir))
    logging.info("Script finished.")


main()

Fetching topic IDs from category between 2025-01-01 00:00:00+00:00 and 2025-04-14 23:59:59.999999+00:00...
Fetched page 0, 30 topics on page. Total unique topics found so far: 3. Continuing...


KeyboardInterrupt: 